# Load modules

ライブラリ読み込み

In [ ]:
# ライブラリ読み込み
import sys, os
import time
import gc
from datetime import datetime as dt
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import japanize_matplotlib
import pydotplus as pdp

import re # 正規表現

import seaborn as sns

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# 評価関数
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

%matplotlib inline

In [ ]:
# 自作関数
from annnmods import *

In [ ]:
print_usage()

In [ ]:
gc.collect()

# Configuration

## 左寄せにするマジックコマンド

In [ ]:
%%html
<style>
    table{float:left}
    .MathJax{float: left;}
</style>

## データフレームの表示設定

In [ ]:
# データフレームの表示行数、表示列数
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)

# カラム内の文字数。デフォルトは50。
pd.set_option("display.max_colwidth", 100)

## パスの設定

In [ ]:
# ファイルのパス
path = os.getcwd()
# 一応チェンジディレクトリしておく
os.chdir(path)

# Constants

## paths

In [ ]:
input_path = '../input/'
output_path = '../output/'

# Functions

## 数値計算

### 数字を四捨五入で丸める

In [ ]:
def pro_round(num, ndigits=0):
    """
    数字を四捨五入で丸める。

    Args:
        num: int or float
            丸めたい数字。

        ndigits: int, optional(default=0)
            丸めた後の小数部分の桁数。

    Returns:
        rounded: int or float
            丸めた後の数字。
    """
    num *= 10 ** ndigits
    rounded = ( 2* num + 1 ) // 2
    rounded /= 10 ** ndigits

    if ndigits == 0:
        rounded = int(rounded)

    return rounded

### スタージェスの公式

In [ ]:
def sturges_rule(num):
    """
    スタージェスの公式を用いて、
    サンプルサイズから階級(カテゴリ、ビン(bins))の数を計算する。
    公式のTeX表記: \[bins = 1 + \log_2{N} \nonumber \]

    Args:
        num: int
            サンプルサイズ。原則1以上の整数を想定。
    
    Returns:
        n_bins: int
            スタージェスの公式から導かれた適切な階級の数。
    """
    # numが0以下の時は1を返す
    if num <= 0:
        num = 1
        return 1
    
    # スタージェスの公式
    n_bins = int(pro_round(1 + np.log2(num), 0))
    
    return n_bins

# Load data

## データ準備

In [ ]:
# タイタニックデータ
# seabornからロード
titanic_df = sns.load_dataset('titanic')

# Explatory data analysis

## 基本的な情報を確認

In [ ]:
# 上から3行読む
titanic_df.head(3)

In [ ]:
# head()の中を何も指定しないと5行読み込む扱い
titanic_df.head()

In [ ]:
# tail()は最後の5行
titanic_df.tail()

In [ ]:
# (行数, 列数)を確認
titanic_df.shape

## 基本統計量を見る

In [ ]:
# 基本統計量を見る
titanic_df.describe()

## 相関を見る

In [ ]:
corr = titanic_df.corr()
corr

In [ ]:
# ヒートマップで表示も可能
sns.heatmap(corr)

### 相関係数(の絶対値)が高い順に並べる

In [ ]:
# 目的変数
target = 'survived'

In [ ]:
# 相関係数の絶対値が高い順に並べる
corr0 = corr[[target]]
corr0['abs_corr'] = corr0[target].abs()
corr0 = corr0.sort_values(by='abs_corr', ascending=False)
corr0

以下2つは同じ
titanic_df['pclass']
titanic_df.pclass

## 度数を見てみる

### pclass

In [ ]:
titanic_df.pclass.value_counts()

### 性別

In [ ]:
titanic_df.sex.value_counts()

## 各変数の尺度を確認することが重要

参考: http://www.gen-info.osaka-u.ac.jp/MEPHAS/express/express0.html  
  
- 順序尺度以上：順序尺度・間隔尺度・比例尺度のいずれかである場合です  
- 間隔尺度以上：間隔尺度・比例尺度のいずれかである場合です  
  
### 名義尺度
単に区別するために用いられている尺度。  
例えば、血液型でＡ型・Ｂ型・Ｏ型・ＡＢ型を、 それぞれ０・１・２・３と数値に対応させたもの。  
これらの変数の平均値を求めてもまったく意味がありません。  
  
### 順序尺度
大小関係にのみ意味がある尺度。  
例えば、治療効果の判定において、 悪化・不変・改善・著効を、それぞれ-１・０・１・２と数値に対応させたもの。  
平均値は定義できないが中央値は定義できます。  
  
### 間隔尺度
数値の差のみに意味がある尺度。  
「距離尺度」とも呼びます。順序尺度の性質も備えています。  
例えば、温度が10℃から15℃になったときに、50％の温度上昇があったとはいいません。 温度が10℃から15℃になったときも、100℃から105℃になったときも、 ともに５℃の温度上昇です。そして、５℃という数値には意味があります。  
  
### 比例尺度
数値の差とともに数値の比にも意味がある尺度。  
「比尺度」とも呼びます。順序尺度・間隔尺度の性質も備えています。  
例えば、体重は50kgから60kgになったときと、100kgから110kgになったときとは、 同じ10kgの増加であっても、前者は20％増、後者は10％増です。 また、比が定義できるということは絶対零点を持つことと同じことを表します。  

## Excelで保存してみる

to_excel(ファイル名, index=(indexをつけるかどうか))

In [ ]:
titanic_path = input_path + 'titanic_df_190907.xlsx'

In [ ]:
# 保存する
titanic_df.to_excel(titanic_path, index=False)

## Excelを読み込む

In [ ]:
# 読み込む
nakami_df = pd.read_excel(titanic_path, index=False)
nakami_df.head(3)

## 変数の概要を確認する

In [ ]:
target

In [ ]:
var_info_df = check_var_info(nakami_df, target=target, file_name=titanic_path, filecol_is=True, transcol_is=True)
var_info_df

In [ ]:
# var_info_dfのカラム名を日本語にしたい場合
enja_dict = {
    'file_name': 'ファイル名',
    'var_name': '変数名',
    'var_name_ja': '変数名日本語訳',
    'dtype': 'データ型',
    'n_unique': '値の種類数',
    'major_vals': '最多数値',
    'count_of_major': '多数値レコード数',
    'missing_rate': '欠損率',
    'n_exist': '非欠損数',
    'n_missing': '欠損数',
    'n_rows': 'レコード数',
    'mean': '平均値',
    'std': '標準偏差',
    'min': '最小値',
    'med': '中央値',
    'max': '最大値',
    'corr_with_target': '目的変数との相関',
    'abs_corr_with_target': '目的変数との相関_絶対値'
}
var_info_df = var_info_df.rename(columns=enja_dict)
var_info_df

In [ ]:
# 出力してみる
var_info_df.to_excel(output_path + 'titanic_var_info.xlsx', index=False)

# Preprocessing

In [ ]:
df = titanic_df.copy()
df.head(3)

In [ ]:
df.shape

## survivedとaliveが被っているのでaliveカラムを削除

In [ ]:
# 重複した行を削除
df[[target, 'alive']].drop_duplicates()

In [ ]:
df = df.drop('alive', axis=1)
df.head(3)

## 階級に分ける

### 階級に分けるカラムの指定

In [ ]:
cls_cols = ['age', 'fare']

### 階級数をスタージェスの公式で計算

In [ ]:
n_class = sturges_rule(len(df))
n_class

### 階級に分ける

In [ ]:
df, bins = get_classes(
    in_df = df,
    columns = cls_cols,
    n_classes = n_class,
    drop = True
    )

In [ ]:
df.head(3)

In [ ]:
cls_bins_cols = list(bins.keys())
cls_bins_cols

## ダミー変数化

### カテゴリ変数のカラムの指定

In [ ]:
var_info_df.columns.tolist()

In [ ]:
var_name = '変数名'
n_unique = '値の種類数'
tmp = var_info_df[[var_name, n_unique]]
tmp = tmp[tmp[n_unique] <= 10]
tmp

In [ ]:
cate_cols = tmp[var_name].values.tolist()
cate_cols.remove('alive')
cate_cols.remove(target)

In [ ]:
cate_cols += cls_bins_cols
cate_cols

### ダミー変数化

In [ ]:
df.columns.tolist()

In [ ]:
df = pd.get_dummies(df, dummy_na=True, columns=cate_cols)
df.head(3)

In [ ]:
df.shape

In [ ]:
# カラムの確認
df.columns.tolist()

### まとめて前処理する場合

In [ ]:
gc.collect()

## 改めてcheck var infoする

In [ ]:
var_info_df0 = check_var_info(df, target=target, file_name=titanic_path, filecol_is=True, transcol_is=True)

# var_info_dfのカラム名を日本語にしたい場合
enja_dict = {
    'file_name': 'ファイル名',
    'var_name': '変数名',
    'var_name_ja': '変数名日本語訳',
    'dtype': 'データ型',
    'n_unique': '値の種類数',
    'major_vals': '最多数値',
    'count_of_major': '多数値レコード数',
    'missing_rate': '欠損率',
    'n_exist': '非欠損数',
    'n_missing': '欠損数',
    'n_rows': 'レコード数',
    'mean': '平均値',
    'std': '標準偏差',
    'min': '最小値',
    'med': '中央値',
    'max': '最大値',
    'corr_with_target': '目的変数との相関',
    'abs_corr_with_target': '目的変数との相関_絶対値'
}
var_info_df0 = var_info_df0.rename(columns=enja_dict)
var_info_df0

In [ ]:
# 出力してみる
var_info_df0.to_excel(output_path + 'titanic_var_info0.xlsx', index=False)

## 目的変数と説明変数で分ける

In [ ]:
df.columns.tolist()

In [ ]:
ignore_cols = [target]

In [ ]:
data_x = df.drop(labels=ignore_cols, axis=1)
data_y = df[target]

## trainとtestに分割

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, random_state=57, test_size=0.3)

In [ ]:
train_x.head()

In [ ]:
print('train_x.shape:', train_x.shape)
print('test_x.shape:', test_x.shape)
print('train_y.shape:', train_y.shape)
print('test_y.shape:', test_y.shape)

# RandomForest

## create model

In [ ]:
rfc = RandomForestClassifier(bootstrap=True, criterion='gini', max_depth=11, min_samples_leaf=5, n_estimators=100, random_state=57)
rfc.fit(train_x, train_y)

## 寄与度分析

In [ ]:
# 説明変数
features = train_x.columns

# 寄与率
importances = rfc.feature_importances_

In [ ]:
# importances描き方
importance_df = pd.DataFrame({'feature': features, 'importance':  importances})
importance_df.sort_values(by='importance', ascending=True).plot(x='feature',y='importance',kind='barh',figsize=(12,9))
display(importance_df.sort_values(by='importance', ascending=False))

## 木の一つを可視化する

In [ ]:
# 試しに木の一つを視覚化する
estimators = rfc.estimators_
file_name = output_path + 'titanic_rfc_tree_0.png'
dot_data = tree.export_graphviz(estimators[0], # 決定木オブジェクトを一つ指定する
                                out_file=None, # ファイルは介さずにGraphvizにdot言語データを渡すのでNone
                                filled=True, # Trueにすると、分岐の際にどちらのノードに多く分類されたのか色で示してくれる
                                node_ids=True, # ノード番号出力?
                                # proportion=True, # パーセンテージ表記
                                rounded=True, # Trueにすると、ノードの角を丸く描画する。
                                feature_names=features, # これを指定しないとチャート上で特徴量の名前が表示されない
                                class_names=['死亡', '生存'], # これを指定しないとチャート上で分類名が表示されない
                                special_characters=True # 特殊文字を扱えるようにする
                                )
graph = pdp.graph_from_dot_data(dot_data)

# 木を画像ファイルとして出力
graph.write_png(file_name)

# 木を表示する
Image(file_name)

## 評価

機械学習の評価指標  
https://data.gunosy.io/entry/2016/08/05/115345

In [ ]:
# trainの評価
pred_train_y = rfc.predict(train_x)
prob_train_y = rfc.predict_proba(train_x)

print('--------------------------------------------------------------------------')
print('train評価')
train_score_df, train_cm_df = print_clf_score(train_y, pred_train_y)

print('--------------------------------------------------------------------------')

# testの評価
pred_test_y = rfc.predict(test_x)
prob_test_y = rfc.predict_proba(test_x)

print('test評価')
test_score_df, test_cm_df = print_clf_score(test_y, pred_test_y)
print('--------------------------------------------------------------------------')

In [ ]:
train_score_df

In [ ]:
test_score_df

## 混同行列

In [ ]:
train_cm_df

In [ ]:
test_cm_df

## 結果をまとめたdfを作成

### result_dfの作成

In [ ]:
result_df = test_x.copy()
result_df[target] = test_y
result_df['pred_y'] = pred_test_y
result_df['pred_prob'] = prob_test_y.T[1]

result_df.head()

In [ ]:
result_df.shape

In [ ]:
def result_to_unique(result_df, cols_list):
    
    cols_df_list = []
    for col in cols_list:
        cols_df_list.append(result_df[col])
    
    cross_df = pd.crosstab(margins=True, index=cols_df_list, columns=all).reset_index()
    cross_df = cross_df[cols_list + ['All']].rename(columns={'All': 'count'})
    
    del cross_df.columns.name # indexを直す
    cross_df[cols_list] = cross_df[cols_list].astype(str) # 型をstrにする
    
    
    # uniqueのlistを作る
    unique_df = result_df.drop_duplicates(subset=cols_list)
    
    unique_df[cols_list] = unique_df[cols_list].astype(str) # 型をstrにする
    
    # マージする
    out_df = pd.merge(unique_df, cross_df, on=cols_list, how='left')
    
    # ソート
    out_df = out_df.sort_values(by=['pred_prob', 'count'], ascending=[False, False]).reset_index(drop=True)
    out_df['count_累積和'] = np.cumsum(out_df['count'].values)
    out_df['count_累積比'] = out_df['count_累積和'] / out_df['count'].values.sum()
    
    return out_df

In [ ]:
# 自作関数を使って、uniqueにした上、集計する
unique_df = result_to_unique(result_df, features.tolist())
unique_df

In [ ]:
# 見やすくする
unique_df['summary'] = unique_df.apply(lambda row: extract_isone(row, features.tolist()), axis=1)
unique_df

# メモリ使用チェック

In [ ]:
mem_cols = ['Variable Name', 'Memory']
memory_df = pd.DataFrame(columns=mem_cols)

for var_name in dir():
    if not var_name.startswith("_"):
        memory_df = memory_df.append(pd.DataFrame([[var_name, sys.getsizeof(eval(var_name))]], columns=mem_cols))

memory_df = memory_df.sort_values(by='Memory', ascending=False).reset_index(drop=True)
display(memory_df)